In [7]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from tensorflow.keras.callbacks import TensorBoard

In [8]:
from sentence_transformers import SentenceTransformer

# Replace 'sentence-transformers/bert-base-nli-mean-tokens' with your desired model name
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df2 = pd.read_parquet('clean_data.pqt')

In [10]:
df = df2.sample(4000)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
115026,67144af73fba2243,wat meant say rob crybaby tired hearing please...,0,0,0,0,0,0
17013,2ce597e695159f02,unblocked pointing elitest wiki admins,0,0,0,0,0,0
28070,4a4935c967021134,also 170 years quite short even antiquity take...,0,0,0,0,0,0
9367,18dd5d69f24159f6,thanks definately check,0,0,0,0,0,0
43847,7506b56eadf7ff6d,quote directly canceled process requires prese...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
57658,9a4514f855ea0037,understand red lots books equal numbers europe...,0,0,0,0,0,0
122645,90125592f745dbb6,cross platform harassment ok made find login c...,0,0,0,0,0,0
127602,aa79eb5f116271c9,resemblance obvious unless similarity pointed ...,0,0,0,0,0,0
19240,32d7c80310cc180f,totally screwed article template ridiculous butt,1,0,0,0,1,0


In [11]:
embeddings= model.encode(df['comment_text'].tolist())
df['emb'] = embeddings.tolist()

In [ ]:
embeddings.shape

In [ ]:
max_len = df['emb'].apply(len).max()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
sequences = np.array(df['emb'].tolist())
sequences = sequences.reshape(400, 1, 768)
print("Shape of sequences before reshaping:", sequences.shape)

In [ ]:
max_words = 1300  # Number of embeddings per sample
dmsn = 768     # Dimensionality of each embedding


In [ ]:


padded_sequences = pad_sequences(sequences, maxlen=max_words, padding='post', truncating='post')


In [ ]:
X = np.array(padded_sequences)
y = df.drop(['id','comment_text', 'emb'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=77)

In [ ]:
X_train.shape

In [ ]:
# dmsn = len(df['emb'].iloc[0])

# print("Dimensionality of the embeddings:", dmsn)

In [ ]:
# # Reshape X_train and X_test to fit the RNN
# X_train = X_train.reshape(X_train.shape[0], max_words, dmsn)
# X_test = X_test.reshape(X_test.shape[0], max_words, dmsn)

In [ ]:
def history_plots(redux, metric):
    valid = "val_"+metric
    plt.plot(redux.history[metric], label=metric)
    plt.plot(redux.history[valid], label=valid)
    plt.legend()
    plt.show()

In [ ]:
filename= "ST_LSTM"
tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

i = Input(shape=(max_words, dmsn))
x = LSTM(128)(i)
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)
lstm_model2 = Model(i, x) 


In [ ]:
lstm_model2.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
r =lstm_model2.fit(X_train, y_train, epochs=10, batch_size=40, validation_data=(X_test, y_test),callbacks=[tb])

In [ ]:
history_plots(r,'accuracy')

In [ ]:
history_plots(r,'loss')

In [ ]:
history_plots(r,'precision')

In [ ]:
history_plots(r,'recall')

In [ ]:
def f1_derive(hist=r,prc='precision',rcl='recall'):
    vp= 'val_'+prc
    v_re= 'val_'+rcl
    CH_hist= pd.DataFrame(hist.history)
    CH_hist['f1'] =(2*CH_hist[prc]*CH_hist[rcl])/(CH_hist[prc]+CH_hist[rcl])
    CH_hist['val_f1'] =(2*CH_hist[vp]*CH_hist[vp])/(CH_hist[vp]+CH_hist[vp])
    CH_hist.fillna(0,inplace=True)

    plt.plot(CH_hist['f1'], label='f1')
    plt.plot(CH_hist['val_f1'], label='val_f1')
    plt.legend()
    plt.show()

In [ ]:
f1_derive()

In [ ]:
filename= "ST_CNN"
tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten

i = Input(shape=(max_words, dmsn))

# Conv1D layer
x = Conv1D(32, 3, activation='relu')(i)
x = MaxPooling1D(2)(x)

# Flatten layer
x = Flatten()(x)

# Dense layers
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)

# Create the Model
cnn_model = Model(i,x)


In [ ]:
cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
g=cnn_model.fit(X_train, y_train, epochs=10, batch_size=40, validation_data=(X_test, y_test),callbacks=[tb])

In [ ]:
history_plots(g,'accuracy')

In [ ]:
history_plots(g,'loss')

In [ ]:
history_plots(g, 'precision')

In [ ]:
history_plots(g,'recall')

In [ ]:
f1_derive(g)